#### Import Dependencies

In [1]:
import requests
from bs4 import BeautifulSoup
import json, os
import pandas as pd
from tqdm import tqdm
import traceback
import PyPDF2

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None

#### Collect Crypto Data
- Get data for top 5k crypto by marketcap from coinmarketcap.com
- Save data to a file (Prevent usage of Api limit)

In [2]:
def get_top_5k_cryptos() -> pd.DataFrame:
    
    api_key = '1d13c476-343c-4717-b648-8a4a1b939b42'
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key,
    }
    
    params = {
        'start': '1',
        'limit': '5000',  # You can adjust this to get more or fewer listings
        'convert': 'USD',
        'sort': 'market_cap'
    }
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    if response.status_code == 200:
        df = pd.DataFrame(data['data'])
        return df
        print(f'Data Saved..')
    else:
        print(f"Error {response.status_code}: {data['status']['error_message']}")
        
# df = get_top_5k_cryptos()
# df.to_csv('../data/topk_crypto.csv', index=False)
# print('Collected top 5k crypto.')

#### Load the dataset

In [3]:
k = 1000
df = pd.read_csv('../data/topk_crypto.csv').head(k)
print(f'Dimensions of the sataset: {df.shape}')
display(df.sample(1))


Dimensions of the sataset: (1000, 18)


,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote
420,22461,Hashflow,HFT,hashflow,110,2022-10-31T07:02:20.000Z,"['collectibles-nfts', 'decentralized-exchange-dex-token', 'defi', 'gaming', 'interoperability', 'dao', 'ethereum-ecosystem', 'binance-smart-chain', 'dex', 'binance-launchpool', 'coinbase-ventures-portfolio', 'avalanche-ecosystem', 'solana-ecosystem', 'dcg-portfolio', 'dragonfly-capital-portfolio', 'electric-capital-portfolio', 'fabric-ventures-portfolio', 'galaxy-digital-portfolio', 'polygon-ecosystem', 'arbitrum-ecosytem', 'cross-chain', 'optimism-ecosystem', 'jump-crypto', 'dwf-labs-portfolio']",NaN,4.067458e+08,1.000000e+09,False,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ETH', 'slug': 'ethereum', 'token_address': '0xb3999F658C0391d94A37f7FF328F3feC942BcADC'}",421,NaN,NaN,NaN,2024-06-06T04:12:00.000Z,"{'USD': {'price': 0.3134934681213295, 'volume_24h': 6482437.38145102, 'volume_change_24h': -41.6085, 'percent_change_1h': -0.51438079, 'percent_change_24h': -0.35324347, 'percent_change_7d': 5.31047042, 'percent_change_30d': -1.58063837, 'percent_change_60d': -23.96552503, 'percent_change_90d': -35.48488846, 'market_cap': 127512156.10134901, 'market_cap_dominance': 0.0048, 'fully_diluted_market_cap': 313493468.12, 'tvl': None, 'last_updated': '2024-06-06T04:12:00.000Z'}}"


#### Get whitepaper links for each of the topk cryptos

In [4]:
def get_whitepaper_link(slug):
    url = f'https://coinmarketcap.com/currencies/{slug}/'
    response = requests.get(url)
    if response.status_code != 200:
        return None
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    divs = soup.find_all('div', class_='sc-d1ede7e3-0 sc-7f0f401-0 gRSwoF gQoblf')
    
    for div in divs:
        if 'whitepaper' not in div.get_text().lower():
            continue
        
        # Find the whitepaper link within this div
        for a_tag in div.find_all('a', href=True):
            return a_tag['href']
    
    return None

# tqdm.pandas()
# df['whitepaper_link'] = df['slug'].progress_apply(get_whitepaper_link)
# df.to_csv(f'../data/top{k}_crypto_w_whitepaper_link.csv', index=False)


#### Loading dataset with whitepaper links

In [5]:
df = pd.read_csv('../data/top1000_crypto_w_whitepaper_link.csv')
extend_df = pd.read_csv('../data/whitepaper_link_corrections.csv')


df = df.merge(extend_df, on='slug', how='left', suffixes=('', '_extend'))
# Override the 'whitepaper_link' column with the values from 'whitepaper_link_extend'
df['whitepaper_link'] = df['whitepaper_link_extend'].combine_first(df['whitepaper_link'])
# Drop the extended 'whitepaper_link' column
df = df.drop(columns=['whitepaper_link_extend'])

df = df.drop(columns = ['tags', 'quote'])
df = df[df['whitepaper_link'].notna()]
print(f'Shape of dataframe: ', df.shape)
print(f'Number of whitepaper links: ', df.whitepaper_link.nunique())
display(df.head(5))


Shape of dataframe:  (423, 17)
Number of whitepaper links:  413


,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,whitepaper_link
0,1,Bitcoin,BTC,bitcoin,11101,2010-07-13T00:00:00.000Z,21000000.0,1.970808e+07,1.970808e+07,False,NaN,1,NaN,NaN,NaN,2024-06-06T04:12:00.000Z,https://bitcoin.org/bitcoin.pdf
1,1027,Ethereum,ETH,ethereum,9038,2015-08-07T00:00:00.000Z,NaN,1.201491e+08,1.201491e+08,True,NaN,2,NaN,NaN,NaN,2024-06-06T04:12:00.000Z,https://ethereum.org/en/whitepaper/
2,825,Tether USDt,USDT,tether,87800,2015-02-25T00:00:00.000Z,NaN,1.123921e+11,1.150861e+11,True,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ETH', 'slug': 'ethereum', 'token_address': '0xdac17f958d2ee523a2206206994597c13d831ec7'}",3,NaN,NaN,NaN,2024-06-06T04:12:00.000Z,https://assets.ctfassets.net/vyse88cgwfbl/5UWgHMvz071t2Cq5yTw5vi/c9798ea8db99311bf90ebe0810938b01/TetherWhitePaper.pdf
4,5426,Solana,SOL,solana,674,2020-04-10T00:00:00.000Z,NaN,4.599202e+08,5.775657e+08,True,NaN,5,NaN,NaN,NaN,2024-06-06T04:12:00.000Z,https://solana.com/solana-whitepaper.pdf
5,3408,USDC,USDC,usd-coin,19594,2018-10-08T00:00:00.000Z,NaN,3.260113e+10,3.260113e+10,False,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ETH', 'slug': 'ethereum', 'token_address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}",6,NaN,NaN,NaN,2024-06-06T04:12:00.000Z,https://f.hubspotusercontent30.net/hubfs/9304636/PDF/centre-whitepaper.pdf


#### Download Content

In [6]:
# Function to verify PDF structure
def verify_pdf(file_name):
    try:
        with open(file_name, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            # Check if the PDF has at least one page
            if len(reader.pages) > 0:
                return True
            else:
                return False
    except Exception as e:
        print(f'An error occurred while verifying {file_name}. Error: {e}')
        # traceback.print_exc()
        return False
    
# Function to download PDF
def fetch_webpage_content(url, file_name):
    try:
        response = requests.get(url)
        
        if response.status_code == 200:
            content = response.content
            
            if 'pdf' in file_name:
                with open(file_name, 'wb') as file:
                    file.write(content)
                    
                if not verify_pdf(file_name):
                    invalid_file_name = file_name.replace('.pdf', '_invalid.pdf')
                    os.rename(file_name, invalid_file_name)
                
            else:
                soup = BeautifulSoup(content, 'html.parser', from_encoding="iso-8859-1")
                content = soup.get_text(separator='\n')
                
                with open(file_name, 'w') as f:
                    f.write(content)
                
        else:
            print(f'Failed to download {file_name}, with url: {url}')
    except Exception as e:
        print(f'An error occurred while downloading {file_name}, with url: {url}, Error is: {e}')
        
        

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    url = row['whitepaper_link']
    type = 'pdf' if 'pdf' in url else 'txt'
    file_name = f"../data/whitepapers/{index + 1}_{row['slug']}.{type}"
    
    if type == 'txt' and os.path.exists(file_name):
        os.remove(file_name)
    
    if not os.path.exists(file_name):
        fetch_webpage_content(url, file_name)
        
        

    

  5%|▌         | 23/423 [00:06<01:47,  3.74it/s]

Failed to download ../data/whitepapers/29_hedera.txt, with url: https://www.hedera.com/papers


  6%|▌         | 26/423 [00:07<01:40,  3.95it/s]

Failed to download ../data/whitepapers/33_immutable-x.txt, with url: https://support.immutable.com/hc/en-us/articles/4405227590799


  7%|▋         | 28/423 [00:07<01:35,  4.13it/s]

...

 94%|█████████▍| 398/423 [03:04<00:07,  3.20it/s]

An error occurred while verifying ../data/whitepapers/543_memetoon.pdf. Error: EOF marker not found


 96%|█████████▌| 405/423 [03:07<00:07,  2.32it/s]

Failed to download ../data/whitepapers/554_cortex.txt, with url: https://cortexlabs.ai/cortex_2_0_whitepaper_en


100%|██████████| 423/423 [03:17<00:00,  2.15it/s]
